### Regression

In [1]:
import os
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 只使用 GPU 0
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/Mbart-Large-50-Regression.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)

In [2]:
import pandas as pd
logger.debug('=============Mbart-Large-50-Regression instruction tuning Strat!=============')

df = pd.read_csv("/home/oem/qx/in_tuning/KETI_nopreprocessed_labeled_dropdup+seperate_ver0.3.csv")

[2025-04-01 17:39:19,863]::3858661222::DEBUG::=============Mbart-Large-50-Regression instruction tuning Strat!=============


In [3]:
# 데이터 크기 확인
print(df.shape)

(11307, 10)


In [4]:
df = df[['transcription', 'OCEAN']]
print(df.head())

                   transcription  \
0  저희가 지금 봐야 되는 게 설치 장소랑 뭘 팔지네요.   
1            그러니까 계절이 10월이니까 쌀쌀한   
2                          좀 쌀쌀한   
3         날씨 그럼 뭐 각자 좀 생각을 해볼까요?   
4      아니면 그냥 저희끼리 얘기를 하는 게 편하신지   

                                               OCEAN  
0  [0.39583333333333337, 0.7708333333333333, 0.72...  
1  [0.39583333333333337, 0.7708333333333333, 0.72...  
2  [0.7708333333333333, 0.5416666666666666, 0.770...  
3  [0.39583333333333337, 0.7708333333333333, 0.72...  
4  [0.39583333333333337, 0.7708333333333333, 0.72...  


In [5]:
import ast 

# OCEAN 컬럼을 리스트로 변환 후 소숫점 4자리로 반올림
def round_ocean_values(ocean_string):
    ocean_list = ast.literal_eval(ocean_string)  # 문자열을 리스트로 변환
    return [round(value, 4) for value in ocean_list]  # 각 값을 소숫점 4자리로 반올림

In [6]:
# OCEAN 값 처리
df['OCEAN'] = df['OCEAN'].apply(round_ocean_values)

# 확인
print(df[['transcription', 'OCEAN']].head())

                   transcription                                     OCEAN
0  저희가 지금 봐야 되는 게 설치 장소랑 뭘 팔지네요.  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]
1            그러니까 계절이 10월이니까 쌀쌀한  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]
2                          좀 쌀쌀한  [0.7708, 0.5417, 0.7708, 0.8125, 0.3542]
3         날씨 그럼 뭐 각자 좀 생각을 해볼까요?  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]
4      아니면 그냥 저희끼리 얘기를 하는 게 편하신지  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]


In [ ]:
import torch
instruction = "주어진 대화를 보고, OCEAN 다섯 가지 성격 특성을 0에서 1 사이의 점수로 예측하세요."

# Instruction + Transcription 결합
df['input_text'] = df['transcription'].apply(
    lambda x: f"### Instruction:\n{instruction}\n\n### Input:\n{x}\n\n### Response:"
)

# 필요 없는 컬럼 제거
df = df[['input_text', 'OCEAN']]

print(df.head())  # 데이터 확인

                                          input_text  \
0  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
1  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
2  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
3  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   
4  ### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특...   

                                      OCEAN  
0  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  
1  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  
2  [0.7708, 0.5417, 0.7708, 0.8125, 0.3542]  
3  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  
4  [0.3958, 0.7708, 0.7292, 0.8333, 0.4375]  


In [8]:
from datasets import Dataset

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)
dataset = dataset.shuffle(seed=42)  # 데이터 섞기
dataset = dataset.train_test_split(test_size=0.2)  # 80% 학습, 20% 검증

# 데이터 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'OCEAN'],
        num_rows: 9045
    })
    test: Dataset({
        features: ['input_text', 'OCEAN'],
        num_rows: 2262
    })
})


In [9]:
import random
import numpy as np
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# GPU 사용 시 시드 고정
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [10]:
# import torch
# import torch.nn as nn
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# # 选择一个 Seq2Seq 的 XLM-R 类似模型（XLM-R 本身没有 Seq2Seq 版本）
# model_name = "facebook/mbart-large-50"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")


In [11]:
import torch
import torch.nn as nn
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 选择一个 Seq2Seq 的 XLM-R 类似模型（XLM-R 本身没有 Seq2Seq 版本）
model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForSeq2SeqLM

class MbartForRegression(nn.Module):
    def __init__(self, model_name):
        super(MbartForRegression, self).__init__()
        
        # loading the full Seq2Seq model
        self.Mbart = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        
        # adding a regression head
        self.regression_head = nn.Linear(self.Mbart.config.d_model, 5)  # 改为 self.Mbart.config.d_model
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        # getting the encoder outputs
        encoder_outputs = self.Mbart.model.encoder(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = encoder_outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
        regression_output = self.regression_head(last_hidden_state[:, 0, :])


        if labels is not None:
            loss_fn = nn.L1Loss()
            loss = loss_fn(regression_output, labels)
            return {"loss": loss, "logits": regression_output}
        
        return {"logits": regression_output}


In [13]:
import torch
# 토크나이징 함수 정의
def preprocess_data(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=256)
    # 반환할 딕셔너리 생성input_ids,attention_mask
    inputs["labels"] = torch.tensor(examples["OCEAN"], dtype=torch.float32)
    return inputs

In [14]:
# 데이터셋 전처리 적용
train_dataset = dataset["train"].map(preprocess_data, batched=True)
eval_dataset = dataset["test"].map(preprocess_data, batched=True)
# 학습용 데이터 확인
print(train_dataset[0])
print(train_dataset)
print(eval_dataset[0])
print(eval_dataset)

Map:   0%|          | 0/9045 [00:00<?, ? examples/s]

Map:   0%|          | 0/2262 [00:00<?, ? examples/s]

{'input_text': '### Instruction:\n주어진 대화를 보고, OCEAN 다섯 가지 성격 특성을 0에서 1 사이의 점수로 예측하세요.\n\n### Input:\n이렇게 돌아다니다가 주류류로 가서 맥주 한 잔하고\n\n### Response:', 'OCEAN': [0.8125, 0.75, 0.6042, 0.7083, 0.3125], 'input_ids': [250004, 6, 187284, 72022, 10763, 12, 5837, 2211, 4033, 123541, 688, 30861, 4, 91888, 105621, 6685, 152915, 44276, 182686, 133954, 413, 757, 1180, 106, 62657, 367, 202031, 1083, 211048, 44476, 5, 6, 187284, 360, 7077, 12, 47383, 68862, 98781, 49806, 5837, 18354, 18354, 1083, 170115, 98720, 2688, 3103, 101256, 2382, 6, 187284, 92748, 12, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [15]:
print(type(eval_dataset[0]["OCEAN"][0]))
print(eval_dataset)

<class 'float'>
Dataset({
    features: ['input_text', 'OCEAN', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2262
})


In [16]:
import os

os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def my_collate_fn(batch):
    input_texts = [item['input_text'] for item in batch]
    labels = torch.stack([torch.tensor(item['OCEAN']) for item in batch])
    return {"input_text": input_texts, "OCEAN": labels}
from torch.utils.data import DataLoader
eval_dataloader = DataLoader(eval_dataset, batch_size=8,collate_fn=my_collate_fn)

In [18]:
from tqdm import tqdm
def evaluate_ocean(model, tokenizer, eval_dataloader, device):
    import re
    from sklearn.metrics import mean_absolute_error

    model.eval()
    all_test_predictions = []
    all_test_labels = []

    for batch in tqdm(eval_dataloader):
        test_input_texts = batch['input_text']
        test_labels = batch["OCEAN"]
        inputs = tokenizer(test_input_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        
        with torch.no_grad():  
            outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]) 
            predictions = outputs["logits"] 
        all_test_predictions.extend(predictions.cpu().numpy().tolist())
        all_test_labels.extend(test_labels.cpu().numpy().tolist())
    model.train()

    mae_score = mean_absolute_error(all_test_labels, all_test_predictions)
    one_minus_mae = 1 - mae_score
    logger.debug(f"\n🧪 Custom Evaluation — 1 - MAE: {one_minus_mae:.4f}")

    return one_minus_mae

In [19]:
def my_train_collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids']) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) for item in batch])
    labels = torch.stack([torch.tensor(item['labels']) for item in batch])  # 明确 shape
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
import shutil
from transformers import TrainerCallback, TrainerControl, TrainerState

class OceanEvalCallback(TrainerCallback):
    def __init__(self, eval_dataloader, tokenizer, device, output_dir="best_model"):
        self.eval_dataloader = eval_dataloader
        self.tokenizer = tokenizer
        self.device = device
        self.output_dir = output_dir
        self.best_score = -float("inf")

    def on_epoch_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        model = kwargs['model']
        print("\n🔍 Running custom evaluation at end of epoch...")

        score = evaluate_ocean(model, self.tokenizer, self.eval_dataloader, self.device)

        if score > self.best_score:
            self.best_score = score
            logger.debug(f"💾 New best model found with score {score:.4f} — saving to {self.output_dir}")

            
            if not os.path.exists(self.output_dir):
                os.makedirs(self.output_dir)
            else:
                shutil.rmtree(self.output_dir)
                os.makedirs(self.output_dir)
            # save the best model
            torch.save(model.state_dict(), os.path.join(self.output_dir, "pytorch_model.bin"))
            self.tokenizer.save_pretrained(self.output_dir)

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 모델, 데이터셋, 토크나이저 설정
# model_name = "bert-base-uncased"  # 원하는 모델 사용 가능
model = MbartForRegression(model_name)
model.to(device)
# 학습 설정
training_args = TrainingArguments(
    output_dir="./regression_Mbart_results",
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="no",  # 关键设置：禁用 Trainer 自动验证
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=False
    # fp16=True
)

callback = OceanEvalCallback(
    eval_dataloader=eval_dataloader,
    tokenizer=tokenizer,
    device=device,
    output_dir="./regression_Mbart_model"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    callbacks=[callback],
    data_collator=my_train_collate_fn
)
print(train_dataset)
# 모델 학습 시작
trainer.train()
# savefile = "./regression_Mbart/epoch50"
# trainer.save_model(savefile)
# tokenizer.save_pretrained(savefile)

2025-04-01 17:39:27.869070: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 17:39:27.872756: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-01 17:39:27.879519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743496767.891384 3651721 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743496767.894715 3651721 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

Dataset({
    features: ['input_text', 'OCEAN', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9045
})


Step,Training Loss
1000,0.133500
2000,0.131200
3000,0.129300
4000,0.126800
5000,0.123700
6000,0.117600
7000,0.114000
8000,0.108700
9000,0.104100
10000,0.097800



🔍 Running custom evaluation at end of epoch...


100%|██████████| 283/283 [00:04<00:00, 60.03it/s]
[2025-04-01 17:41:25,590]::3881085114::DEBUG::
🧪 Custom Evaluation — 1 - MAE: 0.8714
[2025-04-01 17:41:25,591]::2281002358::DEBUG::💾 New best model found with score 0.8714 — saving to ./regression_Mbart_model


...


🔍 Running custom evaluation at end of epoch...


100%|██████████| 283/283 [00:04<00:00, 61.67it/s]
[2025-04-01 19:12:24,673]::3881085114::DEBUG::
🧪 Custom Evaluation — 1 - MAE: 0.8598



🔍 Running custom evaluation at end of epoch...


100%|██████████| 283/283 [00:04<00:00, 57.54it/s]
[2025-04-01 19:14:18,386]::3881085114::DEBUG::
🧪 Custom Evaluation — 1 - MAE: 0.8599


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'Mbart.lm_head.weight', 'Mbart.model.decoder.embed_tokens.weight', 'Mbart.model.encoder.embed_tokens.weight', 'Mbart.model.shared.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

### load model and tokenizer

In [ ]:
# from transformers import AutoTokenizer, AutoConfig
# def load_model_and_tokenizer(model_class, checkpoint_dir):
#     # 加载模型权重
#     model = model_class(config=AutoConfig.from_pretrained(checkpoint_dir))
#     model.load_state_dict(torch.load(f"{checkpoint_dir}/pytorch_model.bin"))
#     model.eval()  # 设置为推理模式

#     # 加载 tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

#     print(f"✅ 模型与 tokenizer 已加载自: {checkpoint_dir}")
#     return model, tokenizer